In [40]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import json

# Load environment variables
load_dotenv()

class ReviewCode:
    review_comment: str

def get_llm_client():
    """
    Initializes and returns an OpenAI client for structured output.
    """
    return ChatOpenAI(
        model="deepseek/deepseek-chat-v3-0324",  # Or "gpt-3.5-turbo-0125"
        temperature=0,
        openai_api_key=os.getenv("API_KEY"),
        openai_api_base="https://openrouter.ai/api/v1"
    ).with_structured_output(ReviewCode)

def review_code_with_llm(code: str, language: str):
    """
    Calls an LLM to review the given code and returns feedback.

    Args:
        code (str): The code snippet to be reviewed.
        language (str): The type of code ('python', 'docker', 'github_workflow').

    Returns:
        dict: JSON feedback from the LLM.
    """
    system_prompt = f"""
    Analyze the following {language} code and provide structured feedback in JSON format.

    Ensure the response follows this schema:
    - "overall_feedback": High-level summary of the review.
    - "detailed_feedback": A categorized breakdown of issues:
        - "code_quality": Issues related to readability, maintainability, and structure.
        - "performance": Inefficiencies, redundant computations, or better algorithms.
        - "security": Vulnerabilities such as unsafe imports, injection risks, or weak authentication.
        - "error_handling": Issues related to exception handling and robustness.
        - "best_practices": Violations of standard coding conventions.
    - "suggestions": Actionable improvements for each issue category.

    Strictly return only valid JSON output.

    Code:
    ```{language}
    {code}```
    """

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt)
    ])
    
    client = get_llm_client()
    chain = prompt | client
    
    try:
        response = chain.invoke({"code": code})
        return response
        return json.loads(response)  # Ensure valid JSON output
    except Exception as e:
        raise
        return {"error": str(e)}

# Sample test
sample_code = """
def hello_world():
    print("hello")
"""

feedback = review_code_with_llm(sample_code, "python")
print("Code Review Feedback:\n", json.dumps(feedback, indent=4))

Code Review Feedback:
 {
    "overall_feedback": "The code is very simple and functional, but lacks several elements that would make it more robust and maintainable.",
    "detailed_feedback": {
        "code_quality": {
            "issues": [
                "The function name 'hello_world' is clear but could be more descriptive if the context requires it.",
                "The function lacks a docstring, which would help in understanding its purpose."
            ]
        },
        "performance": {
            "issues": [
                "No performance issues identified in this simple function."
            ]
        },
        "security": {
            "issues": [
                "No security issues identified in this simple function."
            ]
        },
        "error_handling": {
            "issues": [
                "The function does not handle any potential errors, though none are obvious in this simple case."
            ]
        },
        "best_practices": {
  